In [1]:
# Import library yang diperlukan

import re
import csv
import pandas as pd
import numpy as np
import json
import os
import string
import nltk
import unicodedata

In [2]:
# Load dataset yang akan dibersihkan

dataset = "shopeexpress_raw.csv"
data = pd.read_csv(dataset)

data

,time,tweet
0,2022-09-24 04:25:40+00:00,"@ShopeeID , kenapa g bisa jemput paket gw send..."
1,2022-09-03 08:08:17+00:00,Shopee express id beneran express bgt wkwkwkkw...
2,2022-08-23 07:55:03+00:00,"Belum lagi kurir antar barang seperti di JNE, ..."
3,2022-08-22 01:51:17+00:00,"@sbtcon shopee express, id express"
4,2022-08-02 05:44:59+00:00,@kpopstorebase rumahku kebetulan deket beberap...
...,...,...
85,2020-11-15 14:03:08+00:00,"@herrysw Klo di tempat saya juaranya Anteraja,..."
86,2020-10-28 03:05:34+00:00,Salah pilih ekspedisi kmrn tu. pke shopee expr...
87,2020-10-16 02:39:49+00:00,Shopee express id guys
88,2020-09-26 04:31:08+00:00,@newsplatter Aku sengaja nyebar antara j&amp;t...


In [3]:
# Menghapus data duplikat

data.drop_duplicates(subset="tweet", keep='first', inplace=True)

data

,time,tweet
0,2022-09-24 04:25:40+00:00,"@ShopeeID , kenapa g bisa jemput paket gw send..."
1,2022-09-03 08:08:17+00:00,Shopee express id beneran express bgt wkwkwkkw...
2,2022-08-23 07:55:03+00:00,"Belum lagi kurir antar barang seperti di JNE, ..."
3,2022-08-22 01:51:17+00:00,"@sbtcon shopee express, id express"
4,2022-08-02 05:44:59+00:00,@kpopstorebase rumahku kebetulan deket beberap...
...,...,...
85,2020-11-15 14:03:08+00:00,"@herrysw Klo di tempat saya juaranya Anteraja,..."
86,2020-10-28 03:05:34+00:00,Salah pilih ekspedisi kmrn tu. pke shopee expr...
87,2020-10-16 02:39:49+00:00,Shopee express id guys
88,2020-09-26 04:31:08+00:00,@newsplatter Aku sengaja nyebar antara j&amp;t...


In [4]:
def lower_text(tweet):
    # Mengubah semua huruf menjadi huruf kecil
    tweet = tweet.lower()

    return tweet

# Mengubah teks menjadi huruf kecil pada setiap baris
data['tweet_lower'] = data['tweet'].apply(lower_text)

data

,time,tweet,tweet_lower
0,2022-09-24 04:25:40+00:00,"@ShopeeID , kenapa g bisa jemput paket gw send...","@shopeeid , kenapa g bisa jemput paket gw send..."
1,2022-09-03 08:08:17+00:00,Shopee express id beneran express bgt wkwkwkkw...,shopee express id beneran express bgt wkwkwkkw...
2,2022-08-23 07:55:03+00:00,"Belum lagi kurir antar barang seperti di JNE, ...","belum lagi kurir antar barang seperti di jne, ..."
3,2022-08-22 01:51:17+00:00,"@sbtcon shopee express, id express","@sbtcon shopee express, id express"
4,2022-08-02 05:44:59+00:00,@kpopstorebase rumahku kebetulan deket beberap...,@kpopstorebase rumahku kebetulan deket beberap...
...,...,...,...
85,2020-11-15 14:03:08+00:00,"@herrysw Klo di tempat saya juaranya Anteraja,...","@herrysw klo di tempat saya juaranya anteraja,..."
86,2020-10-28 03:05:34+00:00,Salah pilih ekspedisi kmrn tu. pke shopee expr...,salah pilih ekspedisi kmrn tu. pke shopee expr...
87,2020-10-16 02:39:49+00:00,Shopee express id guys,shopee express id guys
88,2020-09-26 04:31:08+00:00,@newsplatter Aku sengaja nyebar antara j&amp;t...,@newsplatter aku sengaja nyebar antara j&amp;t...


In [5]:
# Menghapus simbol-simbol yang kurang penting

def remove(tweet):
    #normalize
    tweet = unicodedata.normalize('NFKD', tweet).encode('ASCII', 'ignore').decode('utf-8')
    
    #menghapus username
    tweet = re.sub(r'@\w+', '', tweet)
    
    #menghapus angka
    tweet = re.sub('[0-9]+', '', tweet)

    #menghapus stock market tickers
    tweet = re.sub(r'\$\w*', '', tweet)

    #menghapus RT
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    #menghapus hashtag
    tweet = re.sub(r'#\w+', '', tweet)
    
    #menghapus simbol \n
    tweet = re.sub(r'\n', ' ', tweet)
    
    #menghapus huruf yang berulang
    tweet = re.sub(r'(\w)(\1{2,})', r'\1', tweet)
    
    #menghapus tanda &amp;
    tweet = re.sub(r'&amp;(?!\w+)|(?<=\w)&amp;(?=\w+)|(?<=\w)&amp;$', '', tweet)
    
    #menghapus hyperlink
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    #menghapus koma
    tweet = re.sub(r',', ' ', tweet)
    
    #menghapus whitespace
    tweet = re.sub(r"\s+", " ", tweet)
    
    #menghapus karakter lain yang tidak diinginkan
    tweet = re.sub(r'[^\w\s]', ' ', tweet)
    
    return tweet

data['remove_character'] = data['tweet_lower'].apply(lambda x: remove(x))

data

,time,tweet,tweet_lower,remove_character
0,2022-09-24 04:25:40+00:00,"@ShopeeID , kenapa g bisa jemput paket gw send...","@shopeeid , kenapa g bisa jemput paket gw send...",kenapa g bisa jemput paket gw sendiri gw ben...
1,2022-09-03 08:08:17+00:00,Shopee express id beneran express bgt wkwkwkkw...,shopee express id beneran express bgt wkwkwkkw...,shopee express id beneran express bgt wkwkwkkw...
2,2022-08-23 07:55:03+00:00,"Belum lagi kurir antar barang seperti di JNE, ...","belum lagi kurir antar barang seperti di jne, ...",belum lagi kurir antar barang seperti di jne j...
3,2022-08-22 01:51:17+00:00,"@sbtcon shopee express, id express","@sbtcon shopee express, id express",shopee express id express
4,2022-08-02 05:44:59+00:00,@kpopstorebase rumahku kebetulan deket beberap...,@kpopstorebase rumahku kebetulan deket beberap...,rumahku kebetulan deket beberapa ekspedisi jn...
...,...,...,...,...
85,2020-11-15 14:03:08+00:00,"@herrysw Klo di tempat saya juaranya Anteraja,...","@herrysw klo di tempat saya juaranya anteraja,...",klo di tempat saya juaranya anteraja sicepat ...
86,2020-10-28 03:05:34+00:00,Salah pilih ekspedisi kmrn tu. pke shopee expr...,salah pilih ekspedisi kmrn tu. pke shopee expr...,salah pilih ekspedisi kmrn tu pke shopee expr...
87,2020-10-16 02:39:49+00:00,Shopee express id guys,shopee express id guys,shopee express id guys
88,2020-09-26 04:31:08+00:00,@newsplatter Aku sengaja nyebar antara j&amp;t...,@newsplatter aku sengaja nyebar antara j&amp;t...,aku sengaja nyebar antara jt sicepat ninja sh...


In [6]:
# Normalisasi kata tidak baku

reader = csv.reader(open('slang_word_update.csv', 'r'))
d = {}
for row in reader:
    k,v = row
    d[k] = v
    #print d[k]
pat = re.compile(r"\b(%s)\b" % "|".join(d))
for i in range(len(data)):
    text = data['remove_character'].iloc[i]
    text = pat.sub(lambda m: d.get(m.group()), text)
    #print text
    data['remove_character'].iloc[i]=text

data

,time,tweet,tweet_lower,remove_character
0,2022-09-24 04:25:40+00:00,"@ShopeeID , kenapa g bisa jemput paket gw send...","@shopeeid , kenapa g bisa jemput paket gw send...",kenapa tidak bisa jemput paket saya sendiri ...
1,2022-09-03 08:08:17+00:00,Shopee express id beneran express bgt wkwkwkkw...,shopee express id beneran express bgt wkwkwkkw...,shopee express id benaran express sekali wkwkw...
2,2022-08-23 07:55:03+00:00,"Belum lagi kurir antar barang seperti di JNE, ...","belum lagi kurir antar barang seperti di jne, ...",belum lagi kurir antar barang seperti di jne j...
3,2022-08-22 01:51:17+00:00,"@sbtcon shopee express, id express","@sbtcon shopee express, id express",shopee express id express
4,2022-08-02 05:44:59+00:00,@kpopstorebase rumahku kebetulan deket beberap...,@kpopstorebase rumahku kebetulan deket beberap...,rumahku kebetulan dekat beberapa ekspedisi jn...
...,...,...,...,...
85,2020-11-15 14:03:08+00:00,"@herrysw Klo di tempat saya juaranya Anteraja,...","@herrysw klo di tempat saya juaranya anteraja,...",kalau di tempat saya juaranya anteraja sicepa...
86,2020-10-28 03:05:34+00:00,Salah pilih ekspedisi kmrn tu. pke shopee expr...,salah pilih ekspedisi kmrn tu. pke shopee expr...,salah pilih ekspedisi kemarin itu pakai shope...
87,2020-10-16 02:39:49+00:00,Shopee express id guys,shopee express id guys,shopee express id teman teman
88,2020-09-26 04:31:08+00:00,@newsplatter Aku sengaja nyebar antara j&amp;t...,@newsplatter aku sengaja nyebar antara j&amp;t...,saya sengaja nyebar antara juta sicepat ninja...


In [7]:
# Cek KBBI

from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True)

with open('dataKBBI.json', 'r') as outfile:
    corpus_ = json.load(outfile)
# print(corpus)

def cek_kbbi(kalimat, corpus=[], corpus_=corpus_):
    corpus_ = corpus_+corpus
    kalimat = tknzr.tokenize(kalimat.lower())
    new_kalimat = ""
    for i in kalimat:
        if i in corpus_:
            i = i+" "
            new_kalimat += i
    return new_kalimat.rstrip()

for i, row in data.iterrows():
    kalimat = row["remove_character"]
    kalimat_dalam_kbbi = cek_kbbi(kalimat)
    data.at[i, "kbbi_sentence"] = kalimat_dalam_kbbi
    
data

,time,tweet,tweet_lower,remove_character,kbbi_sentence
0,2022-09-24 04:25:40+00:00,"@ShopeeID , kenapa g bisa jemput paket gw send...","@shopeeid , kenapa g bisa jemput paket gw send...",kenapa tidak bisa jemput paket saya sendiri ...,kenapa tidak bisa paket saya sendiri saya bena...
1,2022-09-03 08:08:17+00:00,Shopee express id beneran express bgt wkwkwkkw...,shopee express id beneran express bgt wkwkwkkw...,shopee express id benaran express sekali wkwkw...,shopee express id benaran express sekali sekar...
2,2022-08-23 07:55:03+00:00,"Belum lagi kurir antar barang seperti di JNE, ...","belum lagi kurir antar barang seperti di jne, ...",belum lagi kurir antar barang seperti di jne j...,belum lagi kurir antar barang seperti di jne j...
3,2022-08-22 01:51:17+00:00,"@sbtcon shopee express, id express","@sbtcon shopee express, id express",shopee express id express,shopee express id express
4,2022-08-02 05:44:59+00:00,@kpopstorebase rumahku kebetulan deket beberap...,@kpopstorebase rumahku kebetulan deket beberap...,rumahku kebetulan dekat beberapa ekspedisi jn...,rumahku kebetulan dekat beberapa ekspedisi jnt...
...,...,...,...,...,...
85,2020-11-15 14:03:08+00:00,"@herrysw Klo di tempat saya juaranya Anteraja,...","@herrysw klo di tempat saya juaranya anteraja,...",kalau di tempat saya juaranya anteraja sicepa...,kalau di tempat saya juaranya anteraja sicepat...
86,2020-10-28 03:05:34+00:00,Salah pilih ekspedisi kmrn tu. pke shopee expr...,salah pilih ekspedisi kmrn tu. pke shopee expr...,salah pilih ekspedisi kemarin itu pakai shope...,salah pilih ekspedisi kemarin itu pakai shopee...
87,2020-10-16 02:39:49+00:00,Shopee express id guys,shopee express id guys,shopee express id teman teman,shopee express id teman teman
88,2020-09-26 04:31:08+00:00,@newsplatter Aku sengaja nyebar antara j&amp;t...,@newsplatter aku sengaja nyebar antara j&amp;t...,saya sengaja nyebar antara juta sicepat ninja...,saya sengaja antara juta sicepat ninja shopee ...


In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ohsas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
!pip install PySastrawi


[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
# Menyiapkan stoplist dan stemmer factory

from nltk.corpus import stopwords
stopwords_indonesia = stopwords.words('indonesian')
words_to_remove = ['akhir', 'akhirnya', 'antar', 'awal', 'baik', 'balik', 'banyak', 'bapak', 'baru',
                  'begitu', 'belum', 'benar', 'berapa', 'bertanya', 'besar', 'betul', 'bisa', 'bisakah', 'boleh',
                  'buat', 'bulan', 'datang', 'dekat', 'depan', 'ditanya', 'ditanyai', 'ditanyakan', 'dulu',
                  'guna', 'hari', 'ibu', 'ingat', 'jangan', 'jangankan', 'janganlah', 'jauh', 'jawab',
                  'jawaban', 'jawabnya', 'jelas', 'kasus', 'kebetulan', 'kecil', 'keluar', 'kembali',
                  'keseluruhan', 'keterlaluan', 'kira', 'kiranya', 'kurang', 'lagi', 'lain', 'lainnya', 'lama',
                  'lanjut', 'lanjutnya', 'lebih', 'lewat', 'mana', 'masalah', 'masalahnya', 'mau', 'memang', 
                  'menanyai', 'menanyakan', 'menjawab', 'menjelaskan', 'menyebutkan', 'naik', 'nyatanya', 'pasti',
                  'penting', 'pertama', 'rasa', 'saja', 'sajalah', 'sampai', 'sampai-sampai', 'sebut', 'segalanya', 
                  'segera', 'sekali', 'sekarang', 'seluruhnya', 'sendiri', 'sering', 'seringnya', 'setengah', 'sudah',
                  'tahu', 'tahun', 'tanya', 'tempat', 'tepat', 'terakhir', 'teringat', 'terlalu', 'tidak', 'usah', 
                  'waktu', 'yakin']

for word in words_to_remove:
    stopwords_indonesia.remove(word)

stopwords_indonesia_new = stopwords_indonesia

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [11]:
# Filtering dan stemming

from nltk.tokenize import TweetTokenizer

#emoticon senang
senang = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

#emoticon sedih
sedih = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])    

emoticons = senang.union(sedih)

def clean_tweets(tweet):
    #tokenize tweet
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    cleanTweets = []
    for word in tweet_tokens:
        if (word not in stopwords_indonesia_new and word not in emoticons and word not in string.punctuation):
            stem_word = stemmer.stem(word)
            cleanTweets.append(stem_word)

    return cleanTweets
  
data['cleanTweets'] = data['kbbi_sentence'].apply(lambda tweets: clean_tweets(tweets))

data

,time,tweet,tweet_lower,remove_character,kbbi_sentence,cleanTweets
0,2022-09-24 04:25:40+00:00,"@ShopeeID , kenapa g bisa jemput paket gw send...","@shopeeid , kenapa g bisa jemput paket gw send...",kenapa tidak bisa jemput paket saya sendiri ...,kenapa tidak bisa paket saya sendiri saya bena...,"[tidak, bisa, paket, sendiri, benar, mau, perg..."
1,2022-09-03 08:08:17+00:00,Shopee express id beneran express bgt wkwkwkkw...,shopee express id beneran express bgt wkwkwkkw...,shopee express id benaran express sekali wkwkw...,shopee express id benaran express sekali sekar...,"[shopee, express, id, benar, express, sekali, ..."
2,2022-08-23 07:55:03+00:00,"Belum lagi kurir antar barang seperti di JNE, ...","belum lagi kurir antar barang seperti di jne, ...",belum lagi kurir antar barang seperti di jne j...,belum lagi kurir antar barang seperti di jne j...,"[belum, lagi, kurir, antar, barang, jne, juta,..."
3,2022-08-22 01:51:17+00:00,"@sbtcon shopee express, id express","@sbtcon shopee express, id express",shopee express id express,shopee express id express,"[shopee, express, id, express]"
4,2022-08-02 05:44:59+00:00,@kpopstorebase rumahku kebetulan deket beberap...,@kpopstorebase rumahku kebetulan deket beberap...,rumahku kebetulan dekat beberapa ekspedisi jn...,rumahku kebetulan dekat beberapa ekspedisi jnt...,"[rumah, betul, dekat, ekspedisi, jnt, sicepat,..."
...,...,...,...,...,...,...
85,2020-11-15 14:03:08+00:00,"@herrysw Klo di tempat saya juaranya Anteraja,...","@herrysw klo di tempat saya juaranya anteraja,...",kalau di tempat saya juaranya anteraja sicepa...,kalau di tempat saya juaranya anteraja sicepat...,"[tempat, juara, anteraja, sicepat, juta, shope..."
86,2020-10-28 03:05:34+00:00,Salah pilih ekspedisi kmrn tu. pke shopee expr...,salah pilih ekspedisi kmrn tu. pke shopee expr...,salah pilih ekspedisi kemarin itu pakai shope...,salah pilih ekspedisi kemarin itu pakai shopee...,"[salah, pilih, ekspedisi, kemarin, pakai, shop..."
87,2020-10-16 02:39:49+00:00,Shopee express id guys,shopee express id guys,shopee express id teman teman,shopee express id teman teman,"[shopee, express, id, teman, teman]"
88,2020-09-26 04:31:08+00:00,@newsplatter Aku sengaja nyebar antara j&amp;t...,@newsplatter aku sengaja nyebar antara j&amp;t...,saya sengaja nyebar antara juta sicepat ninja...,saya sengaja antara juta sicepat ninja shopee ...,"[sengaja, juta, sicepat, ninja, shopee, expres..."


In [12]:
# Menghapus tanda baca

def remove_punctuation(text):
    removed_punct = " ".join([char for char in text if char not in string.punctuation])
    return removed_punct

data['CleanTweets'] = data['cleanTweets'].apply(lambda tweets: remove_punctuation(tweets))

data

,time,tweet,tweet_lower,remove_character,kbbi_sentence,cleanTweets,CleanTweets
0,2022-09-24 04:25:40+00:00,"@ShopeeID , kenapa g bisa jemput paket gw send...","@shopeeid , kenapa g bisa jemput paket gw send...",kenapa tidak bisa jemput paket saya sendiri ...,kenapa tidak bisa paket saya sendiri saya bena...,"[tidak, bisa, paket, sendiri, benar, mau, perg...",tidak bisa paket sendiri benar mau pergi mingg...
1,2022-09-03 08:08:17+00:00,Shopee express id beneran express bgt wkwkwkkw...,shopee express id beneran express bgt wkwkwkkw...,shopee express id benaran express sekali wkwkw...,shopee express id benaran express sekali sekar...,"[shopee, express, id, benar, express, sekali, ...",shopee express id benar express sekali cepat s...
2,2022-08-23 07:55:03+00:00,"Belum lagi kurir antar barang seperti di JNE, ...","belum lagi kurir antar barang seperti di jne, ...",belum lagi kurir antar barang seperti di jne j...,belum lagi kurir antar barang seperti di jne j...,"[belum, lagi, kurir, antar, barang, jne, juta,...",belum lagi kurir antar barang jne juta tik sho...
3,2022-08-22 01:51:17+00:00,"@sbtcon shopee express, id express","@sbtcon shopee express, id express",shopee express id express,shopee express id express,"[shopee, express, id, express]",shopee express id express
4,2022-08-02 05:44:59+00:00,@kpopstorebase rumahku kebetulan deket beberap...,@kpopstorebase rumahku kebetulan deket beberap...,rumahku kebetulan dekat beberapa ekspedisi jn...,rumahku kebetulan dekat beberapa ekspedisi jnt...,"[rumah, betul, dekat, ekspedisi, jnt, sicepat,...",rumah betul dekat ekspedisi jnt sicepat shopee...
...,...,...,...,...,...,...,...
85,2020-11-15 14:03:08+00:00,"@herrysw Klo di tempat saya juaranya Anteraja,...","@herrysw klo di tempat saya juaranya anteraja,...",kalau di tempat saya juaranya anteraja sicepa...,kalau di tempat saya juaranya anteraja sicepat...,"[tempat, juara, anteraja, sicepat, juta, shope...",tempat juara anteraja sicepat juta shopee expr...
86,2020-10-28 03:05:34+00:00,Salah pilih ekspedisi kmrn tu. pke shopee expr...,salah pilih ekspedisi kmrn tu. pke shopee expr...,salah pilih ekspedisi kemarin itu pakai shope...,salah pilih ekspedisi kemarin itu pakai shopee...,"[salah, pilih, ekspedisi, kemarin, pakai, shop...",salah pilih ekspedisi kemarin pakai shopee exp...
87,2020-10-16 02:39:49+00:00,Shopee express id guys,shopee express id guys,shopee express id teman teman,shopee express id teman teman,"[shopee, express, id, teman, teman]",shopee express id teman teman
88,2020-09-26 04:31:08+00:00,@newsplatter Aku sengaja nyebar antara j&amp;t...,@newsplatter aku sengaja nyebar antara j&amp;t...,saya sengaja nyebar antara juta sicepat ninja...,saya sengaja antara juta sicepat ninja shopee ...,"[sengaja, juta, sicepat, ninja, shopee, expres...",sengaja juta sicepat ninja shopee express id e...


In [13]:
# Mengurutkan data berdasarkan A-Z

data.sort_values("CleanTweets", inplace=True)

data

,time,tweet,tweet_lower,remove_character,kbbi_sentence,cleanTweets,CleanTweets
83,2020-12-01 10:25:03+00:00,"@ShopeeID Min, udah nunggu 8 hari barang pesan...","@shopeeid min, udah nunggu 8 hari barang pesan...",administrator sudah menunggu hari barang pesa...,administrator sudah menunggu hari barang pesan...,"[administrator, sudah, tunggu, hari, barang, p...",administrator sudah tunggu hari barang pesan s...
72,2021-02-28 04:55:18+00:00,wtb album only resonance pt 2 departure vers y...,wtb album only resonance pt 2 departure vers y...,wtb album only resonance pt departure vers yan...,album pt yang bisa yang bisa pengiriman shopee...,"[album, pt, bisa, bisa, kirim, shopee, express...",album pt bisa bisa kirim shopee express id exp...
10,2022-05-27 13:02:10+00:00,@sbtcon Kalo pickup di aku sicepat sama jnt ta...,@sbtcon kalo pickup di aku sicepat sama jnt ta...,kalau ambil di saya sicepat sama jnt tetapi k...,kalau ambil di saya sicepat sama jnt tetapi ka...,"[ambil, sicepat, jnt, antar, barang, shopee, e...",ambil sicepat jnt antar barang shopee express ...
15,2022-04-27 21:19:38+00:00,"@areinug Asli, pernah di tanyain juga ke csnya...","@areinug asli, pernah di tanyain juga ke csnya...",asli pernah di tanyakan juga ke csnya antar s...,asli pernah di juga ke antar saja shopee expre...,"[asli, antar, saja, shopee, express, id, expre...",asli antar saja shopee express id express tema...
54,2021-04-09 21:54:06+00:00,"@txtdarionlshop Padahal bagus tuh, j&amp;t exp...","@txtdarionlshop padahal bagus tuh, j&amp;t exp...",padahal bagus itu juta express id express sho...,padahal bagus itu juta express id express shop...,"[bagus, juta, express, id, express, shopee, ex...",bagus juta express id express shopee express t...
...,...,...,...,...,...,...,...
48,2021-05-23 03:05:50+00:00,Tolong @ShopeeID shopee express id dibenahi la...,tolong @shopeeid shopee express id dibenahi la...,tolong shopee express id dibenahi lagi ini ha...,tolong shopee express id lagi ini hari berputa...,"[tolong, shopee, express, id, lagi, hari, puta...",tolong shopee express id lagi hari putar jakar...
16,2022-04-23 13:52:59+00:00,@noodelzip iyaa emang aku selalu dpt kurir yan...,@noodelzip iyaa emang aku selalu dpt kurir yan...,ya memang saya selalu dapat kurir yang sama k...,ya memang saya selalu dapat kurir yang sama ka...,"[ya, memang, kurir, shopee, express, id, expre...",ya memang kurir shopee express id express ante...
78,2021-02-02 05:42:20+00:00,"@piockyjr iyaa emang, shopee express/id expres...","@piockyjr iyaa emang, shopee express/id expres...",ya memang shopee express id express begitu it...,ya memang shopee express id express begitu itu...,"[ya, memang, shopee, express, id, express, beg...",ya memang shopee express id express begitu bel...
75,2021-02-17 01:30:40+00:00,"@yhaankyu_ Iya, dm ajasih kalo gue biasanya di...","@yhaankyu_ iya, dm ajasih kalo gue biasanya di...",ya pesan langsung ajasih kalau saya biasanya ...,ya pesan langsung kalau saya biasanya di jnt s...,"[ya, pesan, langsung, jnt, shopee, express, id...",ya pesan langsung jnt shopee express id express


In [14]:
# Menghapus data duplikat

data.drop_duplicates(subset="CleanTweets", keep='first', inplace=True)

data

,time,tweet,tweet_lower,remove_character,kbbi_sentence,cleanTweets,CleanTweets
83,2020-12-01 10:25:03+00:00,"@ShopeeID Min, udah nunggu 8 hari barang pesan...","@shopeeid min, udah nunggu 8 hari barang pesan...",administrator sudah menunggu hari barang pesa...,administrator sudah menunggu hari barang pesan...,"[administrator, sudah, tunggu, hari, barang, p...",administrator sudah tunggu hari barang pesan s...
72,2021-02-28 04:55:18+00:00,wtb album only resonance pt 2 departure vers y...,wtb album only resonance pt 2 departure vers y...,wtb album only resonance pt departure vers yan...,album pt yang bisa yang bisa pengiriman shopee...,"[album, pt, bisa, bisa, kirim, shopee, express...",album pt bisa bisa kirim shopee express id exp...
10,2022-05-27 13:02:10+00:00,@sbtcon Kalo pickup di aku sicepat sama jnt ta...,@sbtcon kalo pickup di aku sicepat sama jnt ta...,kalau ambil di saya sicepat sama jnt tetapi k...,kalau ambil di saya sicepat sama jnt tetapi ka...,"[ambil, sicepat, jnt, antar, barang, shopee, e...",ambil sicepat jnt antar barang shopee express ...
15,2022-04-27 21:19:38+00:00,"@areinug Asli, pernah di tanyain juga ke csnya...","@areinug asli, pernah di tanyain juga ke csnya...",asli pernah di tanyakan juga ke csnya antar s...,asli pernah di juga ke antar saja shopee expre...,"[asli, antar, saja, shopee, express, id, expre...",asli antar saja shopee express id express tema...
54,2021-04-09 21:54:06+00:00,"@txtdarionlshop Padahal bagus tuh, j&amp;t exp...","@txtdarionlshop padahal bagus tuh, j&amp;t exp...",padahal bagus itu juta express id express sho...,padahal bagus itu juta express id express shop...,"[bagus, juta, express, id, express, shopee, ex...",bagus juta express id express shopee express t...
...,...,...,...,...,...,...,...
48,2021-05-23 03:05:50+00:00,Tolong @ShopeeID shopee express id dibenahi la...,tolong @shopeeid shopee express id dibenahi la...,tolong shopee express id dibenahi lagi ini ha...,tolong shopee express id lagi ini hari berputa...,"[tolong, shopee, express, id, lagi, hari, puta...",tolong shopee express id lagi hari putar jakar...
16,2022-04-23 13:52:59+00:00,@noodelzip iyaa emang aku selalu dpt kurir yan...,@noodelzip iyaa emang aku selalu dpt kurir yan...,ya memang saya selalu dapat kurir yang sama k...,ya memang saya selalu dapat kurir yang sama ka...,"[ya, memang, kurir, shopee, express, id, expre...",ya memang kurir shopee express id express ante...
78,2021-02-02 05:42:20+00:00,"@piockyjr iyaa emang, shopee express/id expres...","@piockyjr iyaa emang, shopee express/id expres...",ya memang shopee express id express begitu it...,ya memang shopee express id express begitu itu...,"[ya, memang, shopee, express, id, express, beg...",ya memang shopee express id express begitu bel...
75,2021-02-17 01:30:40+00:00,"@yhaankyu_ Iya, dm ajasih kalo gue biasanya di...","@yhaankyu_ iya, dm ajasih kalo gue biasanya di...",ya pesan langsung ajasih kalau saya biasanya ...,ya pesan langsung kalau saya biasanya di jnt s...,"[ya, pesan, langsung, jnt, shopee, express, id...",ya pesan langsung jnt shopee express id express


In [15]:
# Menghapus baris yang memiliki string kosong pada kolom tertentu
data_clean = data[data['CleanTweets'] != '']

data_clean

,time,tweet,tweet_lower,remove_character,kbbi_sentence,cleanTweets,CleanTweets
83,2020-12-01 10:25:03+00:00,"@ShopeeID Min, udah nunggu 8 hari barang pesan...","@shopeeid min, udah nunggu 8 hari barang pesan...",administrator sudah menunggu hari barang pesa...,administrator sudah menunggu hari barang pesan...,"[administrator, sudah, tunggu, hari, barang, p...",administrator sudah tunggu hari barang pesan s...
72,2021-02-28 04:55:18+00:00,wtb album only resonance pt 2 departure vers y...,wtb album only resonance pt 2 departure vers y...,wtb album only resonance pt departure vers yan...,album pt yang bisa yang bisa pengiriman shopee...,"[album, pt, bisa, bisa, kirim, shopee, express...",album pt bisa bisa kirim shopee express id exp...
10,2022-05-27 13:02:10+00:00,@sbtcon Kalo pickup di aku sicepat sama jnt ta...,@sbtcon kalo pickup di aku sicepat sama jnt ta...,kalau ambil di saya sicepat sama jnt tetapi k...,kalau ambil di saya sicepat sama jnt tetapi ka...,"[ambil, sicepat, jnt, antar, barang, shopee, e...",ambil sicepat jnt antar barang shopee express ...
15,2022-04-27 21:19:38+00:00,"@areinug Asli, pernah di tanyain juga ke csnya...","@areinug asli, pernah di tanyain juga ke csnya...",asli pernah di tanyakan juga ke csnya antar s...,asli pernah di juga ke antar saja shopee expre...,"[asli, antar, saja, shopee, express, id, expre...",asli antar saja shopee express id express tema...
54,2021-04-09 21:54:06+00:00,"@txtdarionlshop Padahal bagus tuh, j&amp;t exp...","@txtdarionlshop padahal bagus tuh, j&amp;t exp...",padahal bagus itu juta express id express sho...,padahal bagus itu juta express id express shop...,"[bagus, juta, express, id, express, shopee, ex...",bagus juta express id express shopee express t...
...,...,...,...,...,...,...,...
48,2021-05-23 03:05:50+00:00,Tolong @ShopeeID shopee express id dibenahi la...,tolong @shopeeid shopee express id dibenahi la...,tolong shopee express id dibenahi lagi ini ha...,tolong shopee express id lagi ini hari berputa...,"[tolong, shopee, express, id, lagi, hari, puta...",tolong shopee express id lagi hari putar jakar...
16,2022-04-23 13:52:59+00:00,@noodelzip iyaa emang aku selalu dpt kurir yan...,@noodelzip iyaa emang aku selalu dpt kurir yan...,ya memang saya selalu dapat kurir yang sama k...,ya memang saya selalu dapat kurir yang sama ka...,"[ya, memang, kurir, shopee, express, id, expre...",ya memang kurir shopee express id express ante...
78,2021-02-02 05:42:20+00:00,"@piockyjr iyaa emang, shopee express/id expres...","@piockyjr iyaa emang, shopee express/id expres...",ya memang shopee express id express begitu it...,ya memang shopee express id express begitu itu...,"[ya, memang, shopee, express, id, express, beg...",ya memang shopee express id express begitu bel...
75,2021-02-17 01:30:40+00:00,"@yhaankyu_ Iya, dm ajasih kalo gue biasanya di...","@yhaankyu_ iya, dm ajasih kalo gue biasanya di...",ya pesan langsung ajasih kalau saya biasanya ...,ya pesan langsung kalau saya biasanya di jnt s...,"[ya, pesan, langsung, jnt, shopee, express, id...",ya pesan langsung jnt shopee express id express


In [16]:
# Menyimpan hasil cleansing ke dalam file csv

data.to_csv(r'data_shopee_baru2.csv', index=False)